# Objectives

### Transformations

Perform the following tasks:
- data cleaning
- data filtering
- data wrangling
- data enrichment
- data validation


### Analysis

After transformations, the data is analyzed using Tableau. FInd the interactive copy [here]().

In [1]:
# Notebook environment settings
import warnings
import os

#ignoring timezone warning
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

#filter out warnings
warnings.filterwarnings("ignore")

In [18]:
# Required packages and libraries
import pyspark
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, sum as spark_sum
from pyspark.sql.types import *


In [4]:
# Initalize spark session
spark = SparkSession.builder \
    .appName("F-A-O") \
    .config("spark.driver.maxResultSize", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()


In [5]:
# Set the maxToStringFields property
spark.conf.set("spark.sql.debug.maxToStringFields", "100")

In [6]:
# Get the dataset
prod_data = spark\
            .read\
            .options(inferSchema="true", header="true")\
            .csv("Production_Crops_Livestock_E_Africa.csv")

There are columns that do not add direct context to my analysis. I will drop them.

In [7]:
# Drop columns
# drop flag cols
to_drop = [col for col in prod_data.columns if col.endswith("F")]

# drop unnecessary cols
to_drop1 = ['Area Code','Area Code (M49)','Item Code','Item Code (CPC)','Element','Element Code','Unit']

# implement
prod_data = (prod_data\
            .drop(*to_drop, *to_drop1))

In [8]:
# Rename columns
#rename year cols
renamed_cols = [col.replace('Y', '') for col in prod_data.columns]
prod_data = prod_data.toDF(*renamed_cols)

#rename other cols
prod_data = prod_data\
                .withColumnRenamed("Area", "Country")


In [9]:
# Trim column names
prod_data = prod_data.select([col(name).alias(name.strip()) for name in prod_data.columns])

In [10]:
# Melt dataframe
#convert spark to pandas df
prod_data = ps.DataFrame(prod_data)

In [11]:
#melt the pandas df
keep_columns=['Country', 'Item']
prod_data=prod_data.melt(id_vars=keep_columns, var_name='Year',value_name='Weight')

I converted the dataframe to pandas dataframe in order to melt it. I could not get along with unpivoting the dataframe with pyspark. If you know how, kindly reach out.
Now I'll have to covert the dataframe back to spark DF for further processing.

In [12]:
# Convert pandas dataframe to RDD 
r_dd = spark.sparkContext.parallelize(prod_data.values.tolist())

#Schema for the pyspark dataFrame
schema = StructType([
    StructField("Country", StringType(), nullable=False),
    StructField("Item", StringType(), nullable=False),
    StructField("Year", StringType(), nullable=False),
    StructField("Weight", DoubleType(), nullable=False)
])

#create pyspark dataFrame
prod_data = spark.createDataFrame(r_dd, schema=schema)

In [13]:
# Create new column
prod_data = prod_data.withColumn("Category",lit(""))

In [14]:
# Categorize items
grains=['Wheat and products', 'Rice (Milled Equivalent)','Barley and products','Maize and products',
        'Millet and products','Cereals, Other','Cereals - Excluding Beer','Rye and products','Oats',
        'Sorghum and products','Cassava and products','Cereals n.e.c.','Wheat','Maize (corn)','Sorghum','Triticale',
       'Fonio']
vegetables=['Potatoes and products','Vegetables, Other','Starchy Roots','Vegetables','Sweet potatoes','Roots, Other',
            'Onions','Plantains','Pimento','Aquatic Plants','Yams','Potatoes','Roots and Tubers, Total',
           'Eggplants (aubergines)','Cauliflowers and broccoli','Edible roots and tubers with high starch or inulin content, n.e.c., fresh',
           'Mushrooms and truffles','Other vegetables, fresh n.e.c.','Apricots','Roots and Tubers, Total',
           'Tomatoes','Cucumbers and gherkins','Chillies and peppers, green (Capsicum spp. and Pimenta spp.)',
           'Cabbages','Carrots and turnips','Cassava, fresh']
sugars=['Sugar (Raw Equivalent)','Sweeteners, Other','Sugar Crops','Sugar & Sweeteners','Sugar non-centrifugal',
        'Sugar beet','Sugar cane','Molasses','Raw cane or beet sugar (centrifugal only)']
fruits=['Olives-Including Preserved','Tomatoes and products','Oranges, Mandarines','Citrus, Other','Bananas',
        'Apples and products','Pineapples and products','Dates','Grapes and Products-Excluding Wine','Fruits, Other',
        'Fruits - Excluding Wine','Lemons, Limes and products','Grapefruit and products','Pineapples',
        'Plums and sloes','Watermelons','Figs','Pomelos and grapefruits','Other fruits, n.e.c.','Fruit Primary',
       'Tangerines, mandarins, clementines','Mangoes, guavas and mangosteens','Quinces','Grapes']
legumes=['Pulses, Other and products','Coffee and products','Cocoa Beans and products','Pulses','Beans','Peas',
         'Soyabeans','Groundnuts-Shelled Eq','Cocoa beans','Broad beans and horse beans, green','Other beans, green',
        '|Lentils, dry','Okra','Cashew nuts, in shell','Groundnuts, excluding shelled','Vetches','Soya beans',
        'Broad beans and horse beans, dry']
seeds=['Sesame seed', 'Rape and Mustardseed','Palm kernels','Sunflower seed','Cottonseed','Castor oil seeds',
      'Rape or colza seed','Melonseed','Cotton seed']
nuts=['Nuts and products','Almonds, in shell','Coconuts - Incl Copra','Treenuts','Coconuts, in shell']
oils_fats=['Soyabean Oil','Groundnut Oil','Sunflowerseed Oil','Rape and Mustard Oil','Cottonseed Oil','Palm Oil',
           'Sesameseed Oil','Olive Oil','Oilcrops Oil, Other','Oilcrops','Vegetable Oils','Oilcrops, Other',
           'Maize Germ Oil','Coconut Oil','Palmkernel Oil','Ricebran Oil','Fish, Body Oil','Fish, Liver Oil',
           'Animal fats','Fats, Animals, Raw','Olive oil','Rapeseed or canola oil, crude','Cheese (All Kinds)',
           'Olives','Oil of palm kernel','Palm oil','Cattle fat, unrendered','Groundnut oil',
          'Safflower-seed oil, crude','Pig fat, rendered','Goat fat, unrendered','Margarine and shortening']
beverages=['Tea-Including Mate','Wine','Beer','Beverages, Alcoholic','Alcoholic Beverages','Beverages, Fermented',
           'Coffee, green']
spices=['Pepper','Spices, Other','Spices','Cloves','Green garlic']
meat=['Bovine Meat','Mutton & Goat Meat','Meat, Other','Meat','Pigmeat','Game meat, fresh, chilled or frozen',
     'Horse meat, fresh or chilled']
sea_food=['Freshwater Fish','Fish, Seafood','Demersal Fish','Pelagic Fish','Marine Fish, Other','Crustaceans',
          'Cephalopods','Molluscs, Other','Aquatic Animals, Others','Aquatic Products, Other',
          'Meat, Aquatic Mammals']
dairy=['Butter, Ghee','Cream','Milk - Excluding Butter','Infant food','Butter of cow milk','Skim Milk & Buttermilk, Dry',
       'Buttermilk, dry','Cheese from skimmed cow milk','Skim milk of cows','Cheese from whole cow milk',
       'Milk, Total','Raw milk of sheep']
poultry=['Eggs','Poultry Meat','Turkeys','Chickens','Hen eggs in shell, fresh','Poultry Birds']
other_animal_products=['Honey','Bees','Offals','Offals, Edible','Edible offals of horses and other equines,  fresh, chilled or frozen',
                      'Raw hides and skins of sheep or lambs','Other meat n.e.c. (excluding mammals), fresh, chilled or frozen',
                      'Edible offal of cattle, fresh, chilled or frozen','Sheep and Goats','Asses','Raw hides and skins of cattle',
                      'Beeswax','Edible offals of camels and other camelids, fresh, chilled or frozen',
                      'Rabbits and hares','Raw hides and skins of goats or kids','Camels','Swine / pigs']
miscellaneous=['Stimulants','Miscellaneous']
other_plant_products=['Flax, processed but not spun','Unmanufactured tobacco','Sisal, raw','Cotton lint, ginned','Fibre Crops, Fibre Equivalent']

# Pattern matching with regex
grains_regex = '|'.join(grains)
vegetables_regex = '|'.join(vegetables)
sugars_regex= '|'.join(sugars)
fruits_regex = '|'.join(fruits)
legumes_regex = '|'.join(legumes)
seeds_regex = '|'.join(seeds)
nuts_regex = '|'.join(nuts)
oils_fats_regex = '|'.join(oils_fats)
beverages_regex = '|'.join(beverages)
spices_regex = '|'.join(spices)
meat_regex = '|'.join(meat)
sea_food_regex = '|'.join(sea_food)
dairy_regex = '|'.join(dairy)
poultry_regex = '|'.join(poultry)
other_animal_products_regex = '|'.join(other_animal_products)
miscellaneous_regex = '|'.join(miscellaneous)
other_plant_products_regex = '|'.join(other_plant_products)

# Update "Category" column based on the values in "Item" column
prod_data = prod_data.withColumn("Category",
                                 when(col("Item").rlike(grains_regex), lit("Grains"))
                                 .when(col("Item").rlike(vegetables_regex), lit("Vegetables"))
                                 .when(col("Item").rlike(sugars_regex), lit("Sugar"))
                                 .when(col("Item").rlike(fruits_regex), lit("Fruits"))
                                 .when(col("Item").rlike(legumes_regex), lit("Legumes"))
                                 .when(col("Item").rlike(seeds_regex), lit("Seeds"))
                                 .when(col("Item").rlike(nuts_regex), lit("Nuts"))
                                 .when(col("Item").rlike(oils_fats_regex), lit("Oil_Fats"))
                                 .when(col("Item").rlike(beverages_regex), lit("Beverages"))
                                 .when(col("Item").rlike(spices_regex), lit("Spices"))
                                 .when(col("Item").rlike(meat_regex), lit("Meat"))
                                 .when(col("Item").rlike(sea_food_regex), lit("Sea_Food"))
                                 .when(col("Item").rlike(dairy_regex), lit("Dairy"))
                                 .when(col("Item").rlike(poultry_regex), lit("Poultry"))
                                 .when(col("Item").rlike(other_animal_products_regex), lit("Other_Animal_Products"))
                                 .when(col("Item").rlike(miscellaneous_regex), lit("Miscellaneous"))
                                 .when(col("Item").rlike(other_plant_products_regex), lit("Other_Plant_Products"))
                                 .otherwise(col("Category")))

In [ ]:
# Check for null values
null_counts = prod_data\
                .select([spark_sum(col(column).isNull().cast("integer")).alias(column) for column in prod_data.columns])
null_counts_dict = null_counts.first().asDict()
for column, count in null_counts_dict.items():
    print(f"Column '{column}': {count}")

In [20]:
# Save the DF as CSV
prod_data.coalesce(1).write.csv("FAO.csv", header=True)

23/06/05 22:31:07 WARN TaskSetManager: Stage 9 contains a task of very large size (50026 KiB). The maximum recommended task size is 1000 KiB.
